# 01 · Feature Engineering

Pull raw UPRO/SPXU data, validate it, and persist engineered factors for downstream modeling.


**Workflow**
- Ensure `00_environment.ipynb` ran successfully
- Download minute bars (cached under `data/minute_bars.parquet`)
- Compute momentum, volatility, and oscillator features
- Persist the final feature panel to `data/features.parquet`


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from momentum_lib import (
    bootstrap_env,
    load_prices,
    validate_prices,
    compute_features,
)

sns.set_theme(style="darkgrid")
bootstrap_env(Path("../.env"))
print("Environment primed.")

data_dir = Path("../data")
data_dir.mkdir(exist_ok=True)
cache_file = data_dir / "prices.parquet"


In [ ]:
prices = load_prices(cache_file=cache_file, period="30d", interval="5m")
prices = validate_prices(prices)
print(f"Fetched {len(prices):,} rows from {prices.index.min()} to {prices.index.max()}")
prices.head()


In [ ]:
recent = prices.tail(500)
ax = sns.lineplot(data=recent)
ax.set_title("Latest price action (500 samples)")
ax.set_ylabel("Close")
plt.show()
recent.tail()


In [ ]:
features = compute_features(prices)
feature_file = data_dir / "features.parquet"
features.to_parquet(feature_file)
print(f"Feature panel saved to {feature_file} with shape {features.shape}")
features.describe().T.head()


In [ ]:
corr = features.corr(numeric_only=True)
plt.figure(figsize=(10, 6))
sns.heatmap(corr, cmap="RdBu_r", center=0)
plt.title("Feature correlation heatmap")
plt.show()

